In [0]:
import json
import pandas as pd
from pyspark.sql import Row

In [0]:
catalog='operations'
schema= 'finance'
table_name = 'dim_tag'

with open("json/dim_tag_liabilities.json","r") as f:
    qry = json.load(f)

rows = []

for main_category, subcats in qry.items():
    for sub_category, items_list in subcats.items():
        for obj in items_list:
            for item_name, labels in obj.items():
                for label in labels:
                    rows.append({
                        "main_category": main_category,
                        "sub_category": sub_category,
                        "item_category": item_name,
                        "item_label": label
                    })

df = spark.createDataFrame(rows).createOrReplaceTempView('tags_df')

In [0]:
tags_df = spark.sql(f"""
select   sha2(concat_ws('|', item_label), 256) AS tag_key_hash
,bigint(substr(xxhash64(concat_ws('|', item_label)), 1, 18)) AS tag_bigint_key
,main_category as tag_level_1 
,sub_category as tag_level_2 
,item_category as tag_level_3
,item_label as terse_label
from tags_df""")

In [0]:
%sql select distinct * from tags_df

In [0]:
(
    tags_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(f"{catalog}.{schema}.{table_name}")
)